In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

# Define the input features from the given matrices P, Q, and R
P = np.array([
    [(0.8, 0.7, 0.2, 0.1), (0.7, 0.6, 0.2, 0.2), (0.5, 0.6, 0.1, 0.1), (0.2, 0.2, 0.0, 0.1)],
    [(0.4, 0.6, 0.2, 0.2), (0.3, 0.6, 0.2, 0.0), (0.5, 0.7, 0.0, 0.1), (0.5, 0.2, 0.4, 0.3)],
    [(0.2, 0.6, 0.1, 0.1), (0.4, 0.5, 0.1, 0.0), (0.8, 0.6, 0.1, 0.1), (0.3, 0.2, 0.4, 0.4)]
])

Q = np.array([
    [(0.7, 0.6, 0.2, 0.1), (0.6, 0.5, 0.2, 0.2), (0.4, 0.6, 0.2, 0.1), (0.3, 0.3, 0.2, 0.1)],
    [(0.5, 0.4, 0.1, 0.2), (0.4, 0.5, 0.1, 0.0), (0.5, 0.8, 0.0, 0.1), (0.5, 0.1, 0.4, 0.3)],
    [(0.2, 0.7, 0.2, 0.1), (0.4, 0.5, 0.1, 0.0), (0.8, 0.7, 0.1, 0.2), (0.3, 0.2, 0.5, 0.4)]
])

R = np.array([
    [(0.6, 0.6, 0.2, 0.1), (0.5, 0.6, 0.2, 0.2), (0.4, 0.6, 0.3, 0.1), (0.3, 0.4, 0.2, 0.1)],
    [(0.5, 0.3, 0.2, 0.1), (0.4, 0.6, 0.1, 0.0), (0.5, 0.8, 0.1, 0.1), (0.5, 0.2, 0.4, 0.2)],
    [(0.3, 0.7, 0.1, 0.1), (0.4, 0.5, 0.1, 0.2), (0.8, 0.7, 0.2, 0.2), (0.2, 0.2, 0.6, 0.4)]
])

# Calculate value matrices V_P, V_Q, V_R
def calculate_value_matrix(matrix):
    return np.array([[mu + T - I - F for (mu, T, I, F) in row] for row in matrix])

V_P = calculate_value_matrix(P)
V_Q = calculate_value_matrix(Q)
V_R = calculate_value_matrix(R)

# Print value matrices
print("Value Matrix V_P:\n", V_P)
print("Value Matrix V_Q:\n", V_Q)
print("Value Matrix V_R:\n", V_R)

# Calculate score matrix S = V_P + V_Q + V_R
S = V_P + V_Q + V_R

# Print score matrix
print("Score Matrix S:\n", S)

# Calculate total scores for each platform (sum of scores across all experts)
total_scores = S.sum(axis=1)

# Print total scores
print("Total Scores for each platform:\n", total_scores)

# Normalize features for training
scaler = StandardScaler()
X_scaled = scaler.fit_transform(S)
# Split data into training and test sets
# Use a smaller test_size to leave more samples for training
X_train, X_test, y_train, y_test = train_test_split(X_scaled, total_scores, test_size=0.1, random_state=42)

# Perform hyperparameter tuning using GridSearchCV
# Reduce the number of cross-validation folds
param_grid = {
    'hidden_layer_sizes': [(10,), (20,), (10, 10), (20, 20)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive']
}

grid_search = GridSearchCV(MLPRegressor(max_iter=1000, random_state=42), param_grid, cv=2, scoring='neg_mean_squared_error') # Use cv=2
grid_search.fit(X_train, y_train)
# Best parameters from GridSearchCV
best_params = grid_search.best_params_
print("Best parameters from GridSearchCV:", best_params)

# Train the neural network regressor with the best parameters
model = MLPRegressor(**best_params, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Data: {mse:.4f}")

# Predict total scores for all platforms
predicted_scores = model.predict(X_scaled)

# Print predicted scores and rank platforms
platforms = ['W1', 'W2', 'W3']
df = pd.DataFrame({
    'Platform': platforms,
    'Predicted_Score': predicted_scores
})

df_sorted = df.sort_values(by='Predicted_Score', ascending=False)
print("Optimal Web Mapping Platforms Ranking:")
print(df_sorted[['Platform', 'Predicted_Score']])


Value Matrix V_P:
 [[ 1.20000000e+00  9.00000000e-01  9.00000000e-01  3.00000000e-01]
 [ 6.00000000e-01  7.00000000e-01  1.10000000e+00 -5.55111512e-17]
 [ 6.00000000e-01  8.00000000e-01  1.20000000e+00 -3.00000000e-01]]
Value Matrix V_Q:
 [[ 1.   0.7  0.7  0.3]
 [ 0.6  0.8  1.2 -0.1]
 [ 0.6  0.8  1.2 -0.4]]
Value Matrix V_R:
 [[ 0.9  0.7  0.6  0.4]
 [ 0.5  0.9  1.1  0.1]
 [ 0.8  0.6  1.1 -0.6]]
Score Matrix S:
 [[ 3.10000000e+00  2.30000000e+00  2.20000000e+00  1.00000000e+00]
 [ 1.70000000e+00  2.40000000e+00  3.40000000e+00 -1.66533454e-16]
 [ 2.00000000e+00  2.20000000e+00  3.50000000e+00 -1.30000000e+00]]
Total Scores for each platform:
 [8.6 7.5 6.4]
Best parameters from GridSearchCV: {'activation': 'relu', 'hidden_layer_sizes': (10, 10), 'learning_rate': 'adaptive', 'solver': 'sgd'}
Mean Squared Error on Test Data: 0.0541
Optimal Web Mapping Platforms Ranking:
  Platform  Predicted_Score
0       W1         8.832524
1       W2         7.461269
2       W3         6.422918
